# 1、LLMChain的使用

举例1：

In [7]:
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SimpleSequentialChain
from langchain.chains.summarize.refine_prompts import prompt_template
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain_core.prompts import PromptTemplate
from sympy import content

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

prompt = PromptTemplate.from_template(
    template="你是一个数学专家，帮我解决如下数学问题：{question}。",
)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
)

response = chain.invoke(input={
    "question": "1+1等于几？"
})

print(response["text"])

在标准的十进制算术系统中，1 + 1 等于 2。

这是自然数加法的基本结果，也是皮亚诺公理体系中的一个直接推论。如果你是在常规数学语境下提问，答案就是：

**1 + 1 = 2**

当然，在某些特殊情况下（比如不同进制、模运算、逻辑运算或抽象代数结构中），结果可能不同。例如：

- 在二进制中，1 + 1 = 10（但数值上仍等于十进制的 2）；
- 在模 2 运算中，1 + 1 ≡ 0 (mod 2)；
- 在布尔代数中，1 + 1 可能表示逻辑“或”，结果仍为 1（取决于定义）。

但若无特别说明，默认情况下 **1 + 1 = 2**。


# 2、顺序连之SimpleSequentialChain的使用

举例1：

In [1]:
from langchain.chains.llm import LLMChain
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.sequential import SimpleSequentialChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

chain_a_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个精通各领域知识的知名教授。"),
        ("user", "请你尽可能详细的解释一下：{knowledge}。"),
    ]
)

chain_a = LLMChain(
    llm=chat_model,
    prompt=chain_a_template,
    verbose=True,
)

chain_b_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结。"),
        ("user", "这是针对一个提问的完整的解释说明内容：{description}"),
        ("user", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内。"),
    ]
)

chain_b = LLMChain(
    llm=chat_model,
    prompt=chain_b_template,
    verbose=True,
)

full_chain = SimpleSequentialChain(
    chains=[chain_a, chain_b],
    verbose=True,
)


# 说明：针对于SimpleSequentialChain来说，输入变量只能有一个，所以这里的输入变量knowledge会传递给chain_a，chain_a的输出变量description会传递给chain_b，最终输出chain_b的结果
# 同时唯一的输入变量名是input
# response = full_chain.invoke(input={
#     "knowledge": "量子力学的基本原理",
# })
response = full_chain.invoke(input={
    "input": "量子力学的基本原理",
})

print(response)

C:\Users\yzy\AppData\Local\Temp\ipykernel_183472\3141649277.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_a = LLMChain(




> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个精通各领域知识的知名教授。
Human: 请你尽可能详细的解释一下：量子力学的基本原理。

> Finished chain.
当然可以。作为一位长期从事量子物理研究的教授，我很高兴为你系统、深入地讲解**量子力学的基本原理**。量子力学是20世纪初发展起来的一套描述微观世界（如原子、电子、光子等）行为的物理理论，它彻底颠覆了经典物理学的决定论世界观，并构成了现代物理学、化学、材料科学乃至信息科学的基础。

下面我将从**历史背景出发**，逐步阐述其**核心原理**，并辅以必要的数学形式和物理诠释，力求既严谨又易于理解。

---

## 一、历史背景与必要性

在19世纪末，经典物理学（牛顿力学 + 麦克斯韦电磁理论）似乎已能解释一切自然现象。然而，几个关键实验无法用经典理论解释：

1. **黑体辐射问题**：经典理论预测“紫外灾难”（能量在高频发散），而普朗克（1900）提出能量必须**量子化**（\(E = h\nu\)）才能拟合实验。
2. **光电效应**：爱因斯坦（1905）用光量子（光子）解释为何电子逸出只与光频率有关，而非强度。
3. **原子光谱的离散性**：玻尔（1913）提出电子只能在特定轨道上运动，角动量量子化（\(L = n\hbar\)）。

这些现象表明：**微观世界的物理规律与宏观世界截然不同**，需要一套全新的理论框架——这就是量子力学的诞生契机。

---

## 二、量子力学的五大基本原理（公理化表述）

现代量子力学通常以**狄拉克-冯·诺依曼公理体系**为基础，主要包括以下五个核心原理：

---

### 原理一：**量子态由希尔伯特空间中的矢量描述**

- 一个物理系统的**状态**（state）由一个**归一化的复向量** \(|\psi\rangle\) 表示，该向量属于一个**复希尔伯特空间** \(\mathcal{H}\)。
- 这个矢量称为**态矢量**（state vector）或**波函数**（在位置表象下写作 \(\psi(\mathbf{r}, t)\)）。
-

举例2：

In [2]:
from langchain.chains.llm import LLMChain
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain_core.prompts import PromptTemplate
from langchain.chains.sequential import SimpleSequentialChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

template1="""你是个剧作家。给定剧本的标题，你的工作是为这个标题写一个大纲。
Title: {title}
"""
prompt_template1 = PromptTemplate(
    input_variables=["title"],
    template=template1,
)
synopsis_chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template1,
)

template2="""你是《纽约时报》的剧作家。有了剧本的大纲，你的工作就是为剧本写一篇评论剧情大纲：
Synopsis: {synopsis}
"""
prompt_template2 = PromptTemplate(
    input_variables=["synopsis"],
    template=template2,
)
review_chain = LLMChain(
    llm=chat_model,
    prompt=prompt_template2,
)

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain],
    verbose=True,
)

response = overall_chain.invoke(input={
    "input": "日落海滩上的悲剧",
})

print(response)



> Entering new SimpleSequentialChain chain...
**剧本标题：日落海滩上的悲剧**

**类型**：心理惊悚 / 家庭剧情 / 悲剧  
**时长**：约90分钟（单幕结构，时间线紧凑）  
**地点**：美国西海岸一处偏僻、风景如画但人迹罕至的日落海滩  

---

### **核心主题**  
- 表面平静下的家庭裂痕  
- 隐瞒与真相的代价  
- 记忆的不可靠性与创伤的代际传递  
- 自然美景与人性阴暗的强烈反差  

---

### **主要人物**  

1. **艾琳（45岁）**：母亲，表面坚强冷静，实则长期压抑内心痛苦；曾是画家，如今不再作画。  
2. **马可（18岁）**：儿子，敏感、内向，即将离家上大学；对父亲之死抱有未解心结。  
3. **莉娜（42岁）**：艾琳的妹妹，性格直率冲动，此次突然造访海滩小屋，带来一封旧信。  
4. **丹尼尔（已故）**：艾琳的丈夫、马可的父亲，三年前在同一天于这片海滩“意外溺亡”。  

---

### **故事大纲**

#### **第一幕：重访旧地**  
- 日落时分，艾琳和马可重返他们三年前最后一次全家度假的海滩小屋。这是丹尼尔去世三周年的纪念日。  
- 艾琳试图营造温馨氛围，但母子之间气氛紧张、对话疏离。马可不断追问父亲死亡细节，艾琳回避。  
- 莉娜不请自来，声称“有些事不能再瞒下去了”，并交给艾琳一封丹尼尔生前写给她的未寄出的信。  
- 闪回片段穿插：三年前那个同样绚烂的日落，丹尼尔与艾琳激烈争吵后走向海浪……  

#### **第二幕：秘密浮出**  
- 莉娜透露：丹尼尔死前正准备离开家庭，他有了外遇，并打算提出离婚。  
- 马可震惊崩溃，指责母亲一直美化父亲形象，掩盖真相。  
- 艾琳终于崩溃，承认那晚她目睹丹尼尔走向大海，却因愤怒没有阻止——她以为他只是赌气，没想到他会溺水。  
- 更深层的秘密揭露：丹尼尔其实患有严重抑郁症，那次争吵是因为他坦白想结束生命，而艾琳误以为他在威胁。  
- 海滩风暴将至，三人情绪濒临失控。马可独自跑向海边，模仿父亲当年的动作走入海中……  

#### **第三幕：日落中的抉择**  
- 艾琳追入海浪，奋力将马可拉回岸边。两人在湿冷沙滩上相拥痛哭，第一次真正面对彼此的伤痛。

# 3、顺序链之SequentialChain的使用

举例1：

In [5]:
from langchain.chains.llm import LLMChain
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.sequential import SequentialChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

chain_a_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个精通各领域知识的知名教授。"),
        ("user", "请你尽可能详细的解释一下：{knowledge}，并且{action}。"),
    ]
)

chain_a = LLMChain(
    llm=chat_model,
    prompt=chain_a_template,
    verbose=True,
    output_key="chain_a_output"
)

chain_b_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结。"),
        ("user", "这是针对一个提问的完整的解释说明内容：{chain_a_output}"),
        ("user", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在100个字以内。"),
    ]
)

chain_b = LLMChain(
    llm=chat_model,
    prompt=chain_b_template,
    verbose=True,
    output_key="chain_b_output"
)

Seq_chain = SequentialChain(
    chains=[chain_a, chain_b],
    input_variables=["knowledge", "action"],
    output_variables=["chain_a_output", "chain_b_output"],
    verbose=True,
)

response = Seq_chain.invoke({
    "knowledge": "中国足球为什么踢的烂",
    "action": "请给出具体的例子说明",
})

print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个精通各领域知识的知名教授。
Human: 请你尽可能详细的解释一下：中国足球为什么踢的烂，并且请给出具体的例子说明。

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你非常善于提取文本中的重要信息，并做出简短的总结。
Human: 这是针对一个提问的完整的解释说明内容：中国足球长期以来在国际赛场表现不佳，这并非单一原因所致，而是系统性、结构性、文化性和制度性多重因素交织的结果。以下我将从**青训体系、联赛机制、管理体制、足球文化、教育体制、职业环境、历史路径依赖**等多个维度进行深入剖析，并辅以具体实例说明。

---

### 一、青训体系薄弱：根基不牢，地动山摇

**问题核心**：缺乏科学、系统、可持续的青少年培养体系。

- **注册青少年球员数量极少**：  
  据中国足协2022年数据，全国注册青少年球员不足5万人（U18及以下）。相比之下，日本注册青少年球员超过70万，德国超100万。这意味着中国足球的人才池极其狭窄。

- **体教分离严重**：  
  在中国，踢球的孩子往往被分流到体校或职业梯队，与普通教育脱节。一旦未能成为职业球员，学业中断，出路受限。家长普遍不愿让孩子“赌”足球前途。  
  **例子**：2019年，山东鲁能足校一名15岁小球员因文化课成绩差被劝退，最终只能回老家读职高——这种“非此即彼”的路径极大抑制了家庭支持孩子踢球的意愿。

- **青训质量参差不齐**：  
  多数基层教练缺乏专业资质，训练方式陈旧，重体能轻技术、重结果轻过程。  
  **例子**：2017年U19国青队在熊猫杯邀请赛上0-3惨败给吉尔吉斯斯坦，赛后录像显示球员基本传接球失误率极高，控球能力远逊于中亚对手。

---

### 二、职业联赛畸形发展：急功近利，泡沫严重

**问题核心**：中超曾经历“金元足球”狂潮，但未转化为竞技实力。

- **过度依赖外援**：  
  2010年代后期，中超

举例2：

In [6]:
from langchain.chains.llm import LLMChain
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain_core.prompts import PromptTemplate
from langchain.chains.sequential import SequentialChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

first_prompt = PromptTemplate.from_template("把下面的内容翻译成中文：\n\n{content}")
chain_one = LLMChain(
    llm=chat_model,
    prompt=first_prompt,
    verbose=True,
    output_key="Chinese_Review",
)

second_prompt = PromptTemplate.from_template("用一句话总结下面的内容：\n\n{Chinese_Review}")
chain_two = LLMChain(
    llm=chat_model,
    prompt=second_prompt,
    verbose=True,
    output_key="Chinese_Summary",
)

third_prompt = PromptTemplate.from_template("下面的内容是什么语言：\n\n{Chinese_Summary}")
chain_three = LLMChain(
    llm=chat_model,
    prompt=third_prompt,
    verbose=True,
    output_key="Language",
)

fourth_prompt = PromptTemplate.from_template("请使用指定的语言对一下内容进行评论：\n\n{Chinese_Summary}\n\n指定语言：{Language}")
chain_four = LLMChain(
    llm=chat_model,
    prompt=fourth_prompt,
    verbose=True,
    output_key="Comment",
)

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["content"],
    output_variables=["Chinese_Review", "Chinese_Summary", "Language", "Comment"],
    verbose=True,
)

content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments, I would like to recognize Jane Smith(SSN:049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching, Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michaeljohnson@example.com)"

response = overall_chain.invoke({
    "content": content,
})

print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
把下面的内容翻译成中文：

Recently, we welcomed several new team members who have made significant contributions to their respective departments, I would like to recognize Jane Smith(SSN:049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching, Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michaeljohnson@example.com)

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
用一句话总结下面的内容：

近期，我们迎来了几位新团队成员，他们已在各自部门做出了重要贡献。在此，我想特别表彰简·史密斯（社保号：049-45-5928）在客户服务方面的出色表现。简始终获得客户的一致好评。此外，请注意我们的员工福利计划开放登记期即将到来。如有任何疑问或需要协助，请联系人力资源代表迈克尔·约翰逊（电话：418-492-3850，邮箱：michaeljohnson@example.com）。

> Finished chain.


> Ent

In [1]:
from langchain.chains.llm import LLMChain
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain_core.prompts import PromptTemplate
from langchain.chains.sequential import SequentialChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

query_chain = LLMChain(
    llm=chat_model,
    prompt=PromptTemplate.from_template("请模拟查询{product}的市场价格，直接返回一个合理的价格数字（如6999），不要包含任何其他文字或代码。"),
    verbose=True,
    output_key="price",
)

promo_chain = LLMChain(
    llm=chat_model,
    prompt=PromptTemplate.from_template(
        "请为{product}(售价：{price}元)的产品撰写一段促销文案，文案内容要吸引人且简洁明了,要求突出产品卖点，50字左右。"
    ),
    verbose=True,
    output_key="promo_text",
)

sequential_chain = SequentialChain(
    chains=[query_chain, promo_chain],
    input_variables=["product"],
    output_variables=["price", "promo_text"],
    verbose=True,
)

response = sequential_chain.invoke({
    "product": "iPhone 17 Pro Max",
})

print(response)

C:\Users\yzy\AppData\Local\Temp\ipykernel_184084\597303028.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  query_chain = LLMChain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
请模拟查询iPhone 17 Pro Max的市场价格，直接返回一个合理的价格数字（如6999），不要包含任何其他文字或代码。

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
请为iPhone 17 Pro Max(售价：8999元)的产品撰写一段促销文案，文案内容要吸引人且简洁明了,要求突出产品卖点，50字左右。

> Finished chain.

> Finished chain.
{'product': 'iPhone 17 Pro Max', 'price': '8999', 'promo_text': 'iPhone 17 Pro Max震撼登场！A18仿生芯片、4800万主摄、钛金属机身，性能与影像巅峰之作，仅售8999元！'}


# 4、数学链LLMMathChain的使用

In [2]:
from langchain_community.chat_models import ChatTongyi
import os
import dotenv
from langchain.chains import LLMMathChain

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

llm_math = LLMMathChain.from_llm(
    llm=chat_model,
    verbose=True,
)

response = llm_math.invoke("10 ** 3 + 5 * 6 - 8 / 4的结果是多少？")
print(response)



> Entering new LLMMathChain chain...
10 ** 3 + 5 * 6 - 8 / 4的结果是多少？```text
10**3 + 5*6 - 8/4
```
...numexpr.evaluate("10**3 + 5*6 - 8/4")...

Answer: 1028.0
> Finished chain.
{'question': '10 ** 3 + 5 * 6 - 8 / 4的结果是多少？', 'answer': 'Answer: 1028.0'}


# 5、路由链RouterChain

# 6、文档链StuffDocumentsChain